1. Download The Dataset

https://www.kaggle.com/code/kredy10/simple-lstm-for-text-classification/data

2. Import The Required Libraries

In [1]:
import os
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from google.colab import drive

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
#Mount and access drive
drive.mount('/content/drive',force_remount=True)
os.chdir('/content/drive/My Drive')
print("Change successful.")

3.Read The Dataset And Do Pre-Processing

In [ ]:
spam_df = pd.read_csv(filepath_or_buffer='/content/spam.csv', delimiter=',',encoding='latin-1')
spam_df.head()

In [ ]:
#List the column names
spam_df.columns

In [ ]:
#Drop the unnamed columns
spam_df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
spam_df.columns

In [ ]:
#Print the number of rows in the dataset
spam_df.shape

In [2]:
#Get the summary statistics of the dataset
spam_df.describe()

NameError: name 'spam_df' is not defined

In [3]:
#Check for null values
spam_df.isna().sum()

NameError: name 'spam_df' is not defined

In [4]:
#Check for duplicated rows
spam_df.duplicated().sum()

NameError: name 'spam_df' is not defined

In [5]:
#Remove the duplicated rows
spam_df = spam_df.drop_duplicates()
spam_df.duplicated().sum()

NameError: name 'spam_df' is not defined

In [6]:
#Display the count of spam and ham labels and Stratified-split is required
spam_df['v1'].hist(bins=3)

NameError: name 'spam_df' is not defined

In [7]:
def wordcloud_vis(column):
  mostcommon = nltk.FreqDist(spam_df[column]).most_common(100)
  wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(mostcommon))
  fig = plt.figure(figsize=(30,10), facecolor='white')
  plt.imshow(wordcloud) #, interpolation="bilinear")
  plt.axis('off')
  plt.show()

In [8]:
#Plot the word-cloud before removing stopwords, performing lemmatization
wordcloud_vis('v2')

NameError: name 'spam_df' is not defined

In [9]:
#Retain only the letters and spaces
spam_df['alpha_text'] = spam_df['v2'].apply(lambda x: re.sub(r'[^a-zA-Z ]+', '', x.lower()))
spam_df.head()

NameError: name 'spam_df' is not defined

In [10]:
#Remove stop-words
nltk.download('stopwords')
spam_df['imp_text'] = spam_df['alpha_text'].apply(lambda x : ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))]))
spam_df.head()

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


NameError: name 'spam_df' is not defined

In [11]:
#Tokenize the data
def tokenize(data):
  generated_token = list(data.split())
  return generated_token
spam_df['token_text'] = spam_df['imp_text'].apply(lambda x: tokenize(x))
spam_df.head()

NameError: name 'spam_df' is not defined

In [12]:
#Perform lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
def lemmatization(list_of_words):
  lemmatized_list = [lemmatizer.lemmatize(word) for word in list_of_words]
  return lemmatized_list
spam_df['lemmatized_text'] = spam_df['token_text'].apply(lambda x: lemmatization(x))
spam_df.head()

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


NameError: name 'spam_df' is not defined

In [13]:
#Combine the tokens (into sentences) to get the final cleansed data
spam_df['clean'] = spam_df['lemmatized_text'].apply(lambda x: ' '.join(x))
spam_df.head()

NameError: name 'spam_df' is not defined

In [14]:
#Display the word cloud after preprocessing
wordcloud_vis('clean')

NameError: name 'spam_df' is not defined

In [28]:
#Number of unique words in spam and ham
df1 = spam_df.loc[spam_df['v1'] == 'spam']
df2 = spam_df.loc[spam_df['v1'] == 'ham']

spam = set()
df1['clean'].str.lower().str.split().apply(spam.update)
print("Number of unique words in spam", len(spam))

ham = set()
df2['clean'].str.lower().str.split().apply(ham.update)
print("Number of unique words in ham", len(ham))

Number of unique words in spam 2037
Number of unique words in ham 6738


In [29]:
#Find the number of overlapping words between spam and ham labels
print("Number of overlapping words between spam and ham: ", len(spam & ham))

Number of overlapping words between spam and ham:  895


In [30]:
#Maximum number of words in a sentence and Useful for applying padding
spam_df['clean'].apply(lambda x:len(str(x).split())).max()

80

In [31]:
#Prepare the data for training
X = spam_df['clean']
y = spam_df['v1']

In [32]:
#Convert the class labels into integer values
le = LabelEncoder()
y = le.fit_transform(y)
y

array([0, 0, 1, ..., 0, 0, 0])

In [33]:
X.shape

(5169,)

In [34]:
y.shape

(5169,)

In [53]:
#Split the data into train, test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

In [54]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
X_train = tf.keras.utils.pad_sequences(tokenized_train, maxlen=100)
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = tf.keras.utils.pad_sequences(tokenized_test, maxlen=100)

4. Create The Model

In [37]:
#Create a wrapper to add layers to the model
model = Sequential()

5. Add Layers (LSTM, Dense-(Hidden Layers), Output)

In [38]:
model.add(Embedding(1000, output_dim=50, input_length=100))
model.add(LSTM(units=64 , return_sequences = True, dropout = 0.2))
model.add(LSTM(units=32 , dropout = 0.1))
model.add(Dense(units = 64 , activation = 'relu'))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 100, 64)           29440     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 96,081
Trainable params: 96,081
Non-traina

6. Compile The Model

In [39]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

7. Fit The Model

In [58]:
model.fit(X_train, y_train, batch_size=128,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=2)])

Epoch 1/10
28/28 [==============================] - 7s 242ms/step - loss: 0.0057 - accuracy: 0.9986 - val_loss: 0.1574 - val_accuracy: 0.9716
Epoch 2/10
28/28 [==============================] - 7s 240ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 0.1487 - val_accuracy: 0.9738
Epoch 3/10
28/28 [==============================] - 7s 238ms/step - loss: 0.0037 - accuracy: 0.9991 - val_loss: 0.1627 - val_accuracy: 0.9761
Epoch 4/10
28/28 [==============================] - 7s 240ms/step - loss: 0.0043 - accuracy: 0.9989 - val_loss: 0.1641 - val_accuracy: 0.9750


8. Save The Model

In [59]:
model.save('spam-classifier.h5')

9. Test The Model

In [60]:
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test,y_test)[1]*100 , "%")

25/25 [==============================] - 1s 24ms/step - loss: 0.1436 - accuracy: 0.9755
Accuracy of the model on Testing Data is -  97.55154848098755 %
